# 필요 패키지 로드

In [1]:
#필요 패키지 로드

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# 데이터 전처리

In [2]:
#데이터 불러오기
#encoding = 'utf-8' or 'cp949'
data1 = pd.read_csv('data_1121.csv', encoding = 'utf-8')

In [3]:
#데이터 타입 등 데이터 정보 확인
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10977 entries, 0 to 10976
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  10977 non-null  int64 
 1   index       10977 non-null  int64 
 2   icls_cnts   10977 non-null  object
 3   label       10977 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 343.2+ KB


In [4]:
#필요한 column 만 가져오기, value별 개수 확인
data1 = data1[['label','icls_cnts']]
data1['label'].value_counts()

 0    7681
 1    2087
-1    1209
Name: label, dtype: int64

In [5]:
#중복 제거하기(뉴스 기사 내용 중복되는 행들 제거)
data1.drop_duplicates(subset = ['icls_cnts'], inplace = True)
data1['label'].value_counts()

 0    6820
 1    2061
-1    1185
Name: label, dtype: int64

In [6]:
df1 = data1

In [7]:
#결측값 여부 확인
print('결측값 여부:',df1.isnull().values.any())

결측값 여부: False


In [8]:
import re 
#텍스트 정제 함수 : 한글 이외의 문자 제거
def text_cleaning(text):
    #한글 정규표현식으로 한글만 추출
    hangul = re.compile('[^ㄱ-ㅣ가-힣]+')
    result = hangul.sub(' ',text)
    return result

In [9]:
df1['ko_text']=df1['icls_cnts'].apply(lambda x: text_cleaning(x))
df1.head

<bound method NDFrame.head of        label                                          icls_cnts  \
0          0  ‘<b>경찰</b> 대혁신 TF’ 구성···<b>경찰</b>청장 ..‘이태원 핼러윈...   
1          0  "<b>경찰</b> 보고체계 마비" 질타…이상민·행안부 별도 언급 않아 .."이태원...   
2          0  이재명 ..“이태원 참사 <b>경찰</b> 질타한 尹, 세월호 때 박근혜 보는 듯”...   
3          0  “<b>경찰</b> 업무에 대대적인 혁신이 필요하다”며 강하게 질책한 데 따른 조치...   
4          0  대통령 질타한 날…<b>집회</b>현장서 게임·쇼핑한 <b>경찰</b> .. 안전<...   
...      ...                                                ...   
10972      1  경남 합천<b>경찰</b>서, 전화금융사기 예방 금융 직원에 감사장 수여 경남 합천...   
10973      1  ‘베스트 여청<b>수사</b>팀’ 선정 김해서부<b>경찰</b>서(서장 심태환)은 ...   
10974      1  김영환 영주<b>경찰</b>서 부청문감사인권관, 경북<b>경찰</b>청 베스트 팀장...   
10975      0  　완주<b>경찰</b>서(서장 박종삼)는 10일 서장, <b>수사</b>과 직원, ...   
10976      1  <b>경찰</b>, 승용차 절도 10대 추격전 끝 검거 ..[KBS 청주] ..훔친...   

                                                 ko_text  
0       경찰 대혁신 구성 경찰 청장 이태원 핼러윈 참사 전후 경찰 의 부실대응을 강하게 ...  
1       경찰 보고체계 마비 질타 이상민 행안부 별도 언급 않아 이태원 참사 감

In [10]:
#pos tagging 함수 만들기

def get_pos(x):
    tagger = Okt()
    pos = tagger.pos(x)
    pos = ['{}/{}'.format(word,tag) for word, tag in pos]
    return pos

#함수 테스트 
result = get_pos(df1['ko_text'][0])
print(result)

['경찰/Noun', '대/Modifier', '혁신/Noun', '구성/Noun', '경찰/Noun', '청장/Noun', '이태원/Noun', '핼러윈/Noun', '참사/Noun', '전후/Noun', '경찰/Noun', '의/Noun', '부실/Noun', '대응/Noun', '을/Josa', '강하게/Adjective', '질타/Noun', '한/Josa', '지/Verb', '이틀/Noun', '만에/Josa', '경찰/Noun', '청/Noun', '이/Josa', '행정안전부/Noun', '경찰/Noun', '국/Noun', '출범/Noun', '후/Noun', '국무총리/Noun', '소속/Noun', '경찰/Noun', '제/Modifier', '도발/Noun', '전/Modifier', '위원회/Noun', '도/Josa', '경찰/Noun', '제도/Noun', '개선/Noun', '방안/Noun', '을/Josa', '논의/Noun', '중인/Noun', '터/Noun', '여서/Josa', '경찰/Noun', '에/Josa', '대한/Noun', '외과/Noun', '수술/Noun', '이/Josa', '본격/Noun', '화할/Adjective', '것/Noun', '으로/Josa', '보인다/Verb', '경찰/Noun', '간부/Noun', '치안정감/Noun', '윤희/Noun', '근/Modifier', '경찰/Noun', '청장/Noun', '은/Josa', '이/Determiner', '날/Noun', '서울/Noun', '서대문구/Noun', '경찰/Noun', '청/Noun', '에서/Josa']


In [11]:
#countvectorizer로 정수화 진행
index_vectorizer = CountVectorizer(tokenizer = lambda x : get_pos(x))
X = index_vectorizer.fit_transform(df1['ko_text'].tolist())

In [12]:
#shape 확인하기
X.shape

(10066, 28596)

In [13]:
#결과 확인해보기
print(df1['ko_text'][0])
print(X[0])

 경찰 대혁신 구성 경찰 청장 이태원 핼러윈 참사 전후 경찰 의 부실대응을 강하게 질타한 지 이틀 만에 경찰 청이 행정안전부 경찰 국 출범 후 국무총리 소속 경찰 제도발전위원회도 경찰 제도 개선방안을 논의 중인 터여서 경찰 에 대한 외과수술이 본격화할 것으로 보인다 경찰 간부 치안정감 윤희근 경찰 청장은 이날 서울 서대문구 경찰 청에서 
  (0, 1391)	11
  (0, 5883)	1
  (0, 27714)	1
  (0, 2283)	1
  (0, 24154)	2
  (0, 19977)	1
  (0, 27464)	1
  (0, 23746)	1
  (0, 21672)	1
  (0, 19272)	1
  (0, 11735)	1
  (0, 6039)	1
  (0, 19215)	2
  (0, 664)	1
  (0, 23419)	1
  (0, 26961)	1
  (0, 22915)	1
  (0, 19983)	1
  (0, 8321)	1
  (0, 24093)	2
  (0, 19361)	2
  (0, 27592)	1
  (0, 2344)	1
  (0, 24575)	1
  (0, 28302)	1
  :	:
  (0, 809)	1
  (0, 10624)	1
  (0, 4977)	1
  (0, 22808)	1
  (0, 25509)	1
  (0, 17280)	1
  (0, 17134)	1
  (0, 6120)	1
  (0, 18225)	1
  (0, 14613)	1
  (0, 11479)	1
  (0, 28090)	1
  (0, 1140)	1
  (0, 19172)	1
  (0, 11382)	1
  (0, 336)	1
  (0, 24790)	1
  (0, 19154)	1
  (0, 2758)	1
  (0, 19190)	1
  (0, 19360)	1
  (0, 4240)	1
  (0, 13627)	1
  (0, 13547)	1
  (0, 17164)	1


In [14]:
#tfidf transformer를 이용하여 중요도 추출
tfidf_vectorizer = TfidfTransformer()
X = tfidf_vectorizer.fit_transform(X)

In [15]:
#결과 확인
print(X.shape)
print(X[0])

(10066, 28596)
  (0, 28302)	0.09910066279186426
  (0, 28090)	0.19308091319563214
  (0, 27714)	0.15105221060887694
  (0, 27592)	0.1229772823502977
  (0, 27464)	0.1369069287562809
  (0, 26961)	0.041956902179263074
  (0, 25509)	0.16470462617395112
  (0, 24790)	0.18161468046615759
  (0, 24575)	0.1429684048656679
  (0, 24154)	0.18855164784375714
  (0, 24093)	0.1232176838148644
  (0, 23746)	0.0958559765744435
  (0, 23419)	0.15105221060887694
  (0, 22915)	0.12940551619544438
  (0, 22808)	0.10459633570561054
  (0, 21999)	0.12734264980506735
  (0, 21983)	0.1229772823502977
  (0, 21672)	0.15926080461121925
  (0, 21448)	0.11002394187959752
  (0, 19983)	0.15105221060887694
  (0, 19977)	0.10005127856572579
  (0, 19361)	0.07098477707382016
  (0, 19360)	0.0726641561497372
  (0, 19272)	0.07568261598591483
  (0, 19215)	0.05478192560493479
  :	:
  (0, 14613)	0.1707270373406938
  (0, 14214)	0.101379282666542
  (0, 13627)	0.07903678291415002
  (0, 13547)	0.16051215846108838
  (0, 11735)	0.1202420629748325

# 모델 적용

In [16]:
#데이터 셋 분리
from sklearn.model_selection import train_test_split

y = df1['label']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1)
print(X_train.shape)
print(X_test.shape)

(9059, 28596)
(1007, 28596)


In [17]:
#모델 적용
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

lr = LogisticRegression(random_state=0)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
y_pred_probability = lr.predict_proba(X_test)[:,1]

print("정확도:%.2f"% accuracy_score(y_test, y_pred))
print("정밀도:%.3f"% precision_score(y_test,y_pred, average = 'weighted'))
print("재현율:%.3f"% recall_score(y_test, y_pred, average = 'weighted'))
print("F1:%.3f"% f1_score(y_test,y_pred, average = 'weighted'))

정확도:0.81
정밀도:0.814
재현율:0.814
F1:0.804


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# 피클링

In [18]:
#save and pickle the model
import pickle
pickle.dump(lr, open('model.pkl','wb'))

# pickle.dump(obj, file, protocol = None, *, fix_imports = True, buffer_callback = None)

#loading model
model = pickle.load(open('model.pkl', 'rb'))